In [18]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator
from collections import Counter
import dateparser
import tabula

In [21]:
dfs = {}
pattern = r'^[eE]xp\d{2}_\d{2}\.pdf$'

for file in os.listdir():
    if re.match(pattern, file):
        name = datetime.datetime.strptime(file[3:file.find('.')], '%y_%m').strftime('%m-%Y')
        dfs.setdefault(name, tabula.read_pdf(file, pages="all"))

In [57]:
indicators = {}

for key in dfs.keys():
    indicators.setdefault(key, {'Покупка физ лицами': '', 'Продажа физ лицами': ''})
    for table in dfs[key]:
        first_column = table.columns[0]
        if any(['покупка физическими лицами' in str(element).lower() for element in table[first_column].to_list()]):
            index = [i for i,j in enumerate(table[first_column].to_list()) if 'покупка физическими лицами' in str(j).lower()][0] + 1
            indicators[key]['Покупка физ лицами'] = table[first_column][index:].copy()
        elif any(['продажа физическими лицами' in str(element).lower() for element in table[first_column].to_list()]):
            index = [i for i,j in enumerate(table[first_column].to_list()) if 'продажа физическими лицами' in str(j).lower()][0] + 1
            indicators[key]['Продажа физ лицами'] = table[first_column][index:].copy()
                

In [49]:
indicators.keys()

dict_keys(['01-2019', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021'])

In [72]:
indicators_new ={}

for key in indicators.keys():
    indicators_new.setdefault(key, {})
    for key2, val2 in indicators[key].items():
        indicators_new[key].setdefault(key2, {})
        for val in val2:
            if 'количеств' in val.lower():
                indicators_new[key][key2].setdefault('Количество сделок', ''.join([i for i in val if i.isdigit()]))
            elif 'средний' in val.lower():
                indicators_new[key][key2].setdefault('Средний размер сделки', ''.join([i for i in val if i.isdigit()]))
                
            
        



In [74]:
temp_df = {'Дата': [], 'Покупка/Продажа': [], 'Операция': [], 'Показатель': []}

for key in indicators_new.keys():
    for key2, val2 in indicators_new[key].items():
        for key3, val3 in val2.items():
            temp_df['Дата'].append(key)
            temp_df['Покупка/Продажа'].append(key2)
            temp_df['Операция'].append(key3)
            temp_df['Показатель'].append(val3)
    

In [76]:
temp_df = pd.DataFrame(temp_df)

In [78]:
temp_df.to_excel('Рыночные показатели.xlsx')